# Designer en Python

En informatique, pour un problème donné, il existe quantité de solutions différentes, dont les qualités respectives peuvent être comparées.
Dans ce notebook, nous allons voir comment un simple problème de géométrie, la modélisation d'un rectangle à partir de ses dimensions, peut être résolu de diverses façons.

## Approche fonctionnelle

Soient `a` et `b` les dimensions du rectangle.

In [1]:
from math import sqrt

def perimetre_rectangle(a: float, b: float) -> float:
    return 2 * (a + b)

def aire_rectangle(a: float, b: float) -> float:
    return a * b

def diagonale_rectangle(a: float, b: float) -> float:
    return sqrt(a ** 2 + b ** 2)

In [2]:
print(f"Le périmètre du rectangle est {perimetre_rectangle(3., 4.)} m.")
print(f"L'aire du rectangle est {aire_rectangle(3., 4.)} m².")
print(f"La diagonale du rectangle est {diagonale_rectangle(3., 4.)} m.")

Le périmètre du rectangle est 14.0 m.
L'aire du rectangle est 12.0 m².
La diagonale du rectangle est 5.0 m.


## Une première approche objet

Dans l'approche fonctionnelle, les trois fonctions partagent des points communs :

* Elles sont suffixées `_rectangle` pour les différencier de fonctions similaires qui s'appliqueraient à d'autres géométries.
* Elles prennent les mêmes paramètres `a` et `b` en paramètre.

Une approche objet paraît adaptée.

In [3]:
from math import sqrt

class Rectangle:

    def __init__(self, a: float, b: float):
        self.a = a
        self.b = b

    def perimetre(self) -> float:
        return 2 * (self.a + self.b)

    def aire(self) -> float:
        return self.a * self.b

    def diagonale(self) -> float:
        return sqrt(self.a ** 2 + self.b ** 2)

Cette modélisation est très simple et très efficace.
Périmètre, aire et diagonale ne seront calculés, comme dans l'approche fonctionnelle, que si l'utilisateur de la classe `Rectangle` appelle les méthodes correspondantes.
Du point de vue de l'occupation mémoire, chaque instance de `Rectangle` est entièrement définie par `self.a` et `self.b`.

In [4]:
r = Rectangle(3., 4.)
print(f"Le périmètre du rectangle est {r.perimetre()} m.")
print(f"L'aire du rectangle est {r.aire()} m².")
print(f"La diagonale du rectangle est {r.diagonale()} m.")

Le périmètre du rectangle est 14.0 m.
L'aire du rectangle est 12.0 m².
La diagonale du rectangle est 5.0 m.


## Optimiser le temps d'exécution

En supposant qu'on accède souvent au périmètre, à l'aire et à la diagonale d'un rectangle dont les dimensions changent rarement, l'intuition conduit à l'idée de les précalculer, pour [optimiser](https://fr.wikipedia.org/wiki/Optimisation_de_code) le temps d'exécution.

In [5]:
from math import sqrt

class Rectangle:

    def __init__(self, a: float, b: float):
        self.a = a
        self.b = b
        self.perimetre = 2 * (self.a + self.b)
        self.aire = self.a * self.b
        self.diagonale = sqrt(self.a ** 2 + self.b ** 2)

Cette implémentation est en réalité beaucoup plus fragile que la première.
En particulier, l'[encapsulation](https://fr.wikipedia.org/wiki/Encapsulation_(programmation)) a été perdue.
Plusieurs familles de bugs peuvent se produire.

In [6]:
r = Rectangle(3., 4.)
print(f"Le périmètre du rectangle est {r.perimetre} m.")

r.a = 1.
print(f"Le périmètre du rectangle est {r.perimetre} m.")

Le périmètre du rectangle est 14.0 m.
Le périmètre du rectangle est 14.0 m.


En particulier la modification des dimensions n'entraîne pas la modification du périmètre.

## Définir des propriétés

Pour restaurer l'encapsulation, il faut introduire la notion de [propriété](https://docs.python.org/fr/3.15/library/functions.html#property).
Sommairement, une propriété permet d'encapsuler un attribut à l'aide d'un accesseur et d'un mutateur (_getter_ et _setter_ en anglais).

Du point de vue du code, rien ne distingue un accesseur ou un mutateur d'une méthode ordinaire.
En revanche, du point de vue du développeur, ils jouent un rôle sémantique fondamental :

* Un accesseur permet d'accéder à un attribut/propriété sans [effet de bord](https://fr.wikipedia.org/wiki/Effet_de_bord_(informatique)).
* Un mutateur modifie un attribut/propriété et ne retourne pas de valeur.

⚠️ L'encapsulation en Python repose sur des conventions.
Un attribut est considéré privé s'il commence par un `_`.
Un attribut privé ne devrait jamais être manipulé hors de sa classe.

In [7]:
from math import sqrt

class Rectangle:

    def __init__(self, a: float, b: float):
        self._a = a
        self._b = b
        self._recalculer()

    def _recalculer(self):
        self._perimetre = 2 * (self._a + self._b)
        self._aire = self._a * self._b
        self._diagonale = sqrt(self._a ** 2 + self._b ** 2)

    def get_a(self) -> float:
        return self._a

    def set_a(self, a: float):
        self._a = a
        self._recalculer()

    def get_b(self) -> float:
        return self._b

    def set_b(self, b: float):
        self._b = b
        self._recalculer()

    def get_perimetre(self) -> float:
        return self._perimetre

    def get_aire(self) -> float:
        return self._aire

    def get_diagonale(self) -> float:
        return self._diagonale

    a = property(get_a, set_a)
    b = property(get_b, set_b)
    perimetre = property(get_perimetre)
    aire = property(get_aire)
    diagonale = property(get_diagonale)

Le nombre de lignes de code a significativement augmenté !
En termes de maintenabilité, on peut se demander si précalculer périmètre, aire et diagonale était une si bonne idée.
Comme disait [Dijkstra](https://fr.wikipedia.org/wiki/Edsger_Dijkstra) : "L'optimisation prématurée est la source de tous les maux."

In [8]:
r = Rectangle(3., 4.)
print(f"Le périmètre du rectangle est {r.perimetre} m.")

r.a = 1.
print(f"Le périmètre du rectangle est {r.perimetre} m.")
print(f"L'aire du rectangle est {r.aire} m².")
print(f"La diagonale du rectangle est {r.diagonale} m.")

Le périmètre du rectangle est 14.0 m.
Le périmètre du rectangle est 10.0 m.
L'aire du rectangle est 4.0 m².
La diagonale du rectangle est 4.123105625617661 m.


En l'absence de mutateur, périmètre, aire et diagonale ne sont pas modifiables.

In [9]:
r.perimetre = 10

AttributeError: property 'perimetre' of 'Rectangle' object has no setter

## Une syntaxe plus concise avec les décorateurs

Les [décorateurs](https://docs.python.org/fr/3.15/glossary.html#term-decorator) permettent d'obtenir le même comportement, avec une syntaxe allégée.

In [10]:
from math import sqrt

class Rectangle:

    def __init__(self, a: float, b: float):
        self._a = a
        self._b = b
        self._recalculer()

    def _recalculer(self):
        self._perimetre = 2 * (self._a + self._b)
        self._aire = self._a * self._b
        self._diagonale = sqrt(self._a ** 2 + self._b ** 2)

    @property
    def a(self) -> float:
        return self._a

    @a.setter
    def a(self, new_a: float):
        self._a = new_a
        self._recalculer()

    @property
    def b(self) -> float:
        return self._b

    @b.setter
    def b(self, new_b: float):
        self._b = new_b
        self._recalculer()

    @property
    def perimetre(self) -> float:
        return self._perimetre

    @property
    def aire(self) -> float:
        return self._aire

    @property
    def diagonale(self) -> float:
        return self._diagonale

## Immu(t)abilité

Jusqu'ici, les designs présentés permettent de modifier `a` et `b`.
Toutefois, est-ce réellement un besoin ?
D'autre part, d'un point de vue sémantique, est-ce que changer une dimension d'un rectangle ne revient pas, au fond, à changer _de_ rectangle ?

In [11]:
from math import sqrt

class Rectangle:

    def __init__(self, a: float, b: float):
        self._a = a
        self._b = b
        self._perimetre = 2 * (self._a + self._b)
        self._aire = self._a * self._b
        self._diagonale = sqrt(self._a ** 2 + self._b ** 2) 

    @property
    def a(self) -> float:
        return self._a

    def with_a(self, new_a: float) -> Rectangle:
        return Rectangle(new_a, self.b)

    @property
    def b(self) -> float:
        return self._b

    def with_b(self, new_b: float) -> Rectangle:
        return Rectangle(self.a, new_b)

    @property
    def perimetre(self) -> float:
        return self._perimetre

    @property
    def aire(self) -> float:
        return self._aire

    @property
    def diagonale(self) -> float:
        return self._diagonale

Le design ci-dessus produit une classe `Rectangle` [immuable](https://fr.wikipedia.org/wiki/Objet_immuable).

Les méthodes `with_a(float)` et `with_b(float)` permettent de construire un nouveau `Rectangle` en spécifiant une nouvelle dimension.
Ces méthodes sont parfaitement optionnelles.

In [12]:
r = Rectangle(3., 4.)
print(f"Le périmètre du rectangle est {r.perimetre} m.")

r = r.with_a(1.)
print(f"Le périmètre du rectangle est {r.perimetre} m.")
print(f"L'aire du rectangle est {r.aire} m².")
print(f"La diagonale du rectangle est {r.diagonale} m.")

Le périmètre du rectangle est 14.0 m.
Le périmètre du rectangle est 10.0 m.
L'aire du rectangle est 4.0 m².
La diagonale du rectangle est 4.123105625617661 m.


In [13]:
r.a = 1.

AttributeError: property 'a' of 'Rectangle' object has no setter

Dans ce design, `a` et `b` ne sont plus modifiables une fois le `Rectangle` créé.

## Validation

Jusqu'à présent, les dimensions fournies par l'utilisateur de la classe `Rectangle` n'ont pas été questionnées.
Le prochain design va assurer que `a` et `b` sont forcément positifs. 

In [14]:
from math import sqrt

class Rectangle:

    def __init__(self, a: float, b: float):
        self._set_a(a)
        self._set_b(b)

    @property
    def a(self) -> float:
        return self._a

    @a.setter
    def a(self, a: float):
        self._set_a(a)

    def _set_a(self, a: float):
        if not a > 0:
            raise ValueError("a must be positive")
        self._a = a
    
    @property
    def b(self) -> float:
        return self._b

    @b.setter
    def b(self, b: float):
        self._set_b(b)

    def _set_b(self, b: float):
        if not b > 0:
            raise ValueError("b must be positive")
        self._b = b

    def perimetre(self) -> float:
        return 2 * (self._a + self._b)

    def aire(self) -> float:
        return self._a * self._b

    def diagonale(self) -> float:
        return sqrt(self._a ** 2 + self._b ** 2)

In [15]:
r = Rectangle(3., -4.)

ValueError: b must be positive

Il n'est plus possible de créer un `Rectangle` avec des dimensions négatives : une [erreur](https://docs.python.org/3/library/exceptions.html#ValueError) est levée.

In [16]:
r = Rectangle(3., 4.)
r.a = -1.

ValueError: a must be positive

On le voit, un même problème, même trivial, trouve de multiples solutions avec Python.
Chacune a ses forces et ses faiblesses.
Le choix dépend de beaucoup de critères : besoins fonctionnels, envergure du produit, maturité de l'équipe.
Comme souvent, [il n'y a pas de baguette magique](https://fr.wikipedia.org/wiki/Pas_de_balle_en_argent).
